In [ ]:
%config IPCompleter.greedy=True

In [ ]:
# zipfile.ZipFile
# os.path.exists, shutil.rmtree, os.makedirs
# random.sample, copyfile(this_file, destination)
# optimizer=RMSprop, ImageDataGenerator(rescale=1.0/255.)
# acc, val_acc, loss, val_loss, plt.plot, plt.title, plt.figure()

In [ ]:
import zipfile
zipRef = zipfile.ZipFile('/tmp/horse-or-human.zip', 'r')
zipRef.extractall('/tmp/horse_or_human/')
zipRef.close()

In [ ]:
import os
import shutil

def create_directory(path):
    if(os.path.exists(path)):
        shutil.rmtree(path)
    else:
        os.makedirs(path)

source_human_path = '/tmp/horse_or_human/humans/'
train_human_path = '/tmp/horses_v_humans/training/humans/'
test_human_path = '/tmp/horses_v_humans/testing/humans/'

source_horse_path = '/tmp/horse_or_human/horses/'
train_horse_path = '/tmp/horses_v_humans/training/horses/'
test_horse_path = '/tmp/horses_v_humans/testing/horses/'

create_directory(train_human_path)
create_directory(test_human_path)

create_directory(train_horse_path)
create_directory(test_horse_path)

In [ ]:
import os
import random
from shutil import copyfile

def split_data(source, train_path, test_path, size = 0.9):
    files = []
    for filename in os.listdir(source):
        this_file = source + filename
        if os.path.getsize(this_file) > 0:
            files.append(filename)
        else:
            print(filename + ' is zero length, so ignoring.')
    
    shuffled_files = random.sample(files, len(files))
    train_files = shuffled_files[0: int(len(shuffled_files) * size)]
    test_files = shuffled_files[-(len(shuffled_files) - len(train_files)):]
    
    for filename in train_files:
        this_file = source + filename
        target_file = train_path + filename
        copyfile(this_file, target_file)
    
    for filename in test_files:
        this_file = source + filename
        target_file = test_path + filename
        copyfile(this_file, target_file)

split_data(source_human_path, train_human_path, test_human_path, 0.9)
split_data(source_horse_path, train_horse_path, test_horse_path, 0.9)

In [ ]:
import os
train_horse_names = os.listdir(train_horse_path)
print(train_horse_names[:10])

train_human_names = os.listdir(train_human_path)
print(train_human_names[:10])

In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

# Index for iterating over images
pic_index = 0
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_horse_pix = [os.path.join(train_horse_path, fname) 
                for fname in train_horse_names[pic_index-8:pic_index]]
next_human_pix = [os.path.join(train_human_path, fname) 
                for fname in train_human_names[pic_index-8:pic_index]]

for i, img_path in enumerate(next_horse_pix+next_human_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1.0/255.0) 
train_gen = train_datagen.flow_from_directory('/tmp/horses_v_humans/training/', batch_size=100, class_mode='binary', target_size=(150,150))
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_gen = train_datagen.flow_from_directory('/tmp/horses_v_humans/testing/', batch_size=100, class_mode='binary', target_size=(150,150))


In [ ]:
import tensorflow as tf
from tensorflow import keras

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(tf.keras.layers.MaxPooling2D(2, 2))
# The second convolution
model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2,2))
# The third convolution
model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2,2))
# The fourth convolution
model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2,2))
# The fifth convolution
model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2,2))
    
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(24, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=RMSprop(0.001), loss='binary_crossentropy', metrics=['acc'])

In [ ]:
history = model.fit_generator(train_gen, epochs=5, verbose=1, validation_data=test_gen)

In [ ]:
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
epochs = range(len(acc))

plt.plot(epochs, acc, 'r', 'Train accuracy')
plt.plot(epochs, val_acc, 'b', 'Validation accuracy')
plt.title('Train - Validation accuracy')
plt.figure()

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(epochs, loss, 'r', 'Train loss')
plt.plot(epochs, val_loss, 'b', 'Validation loss')
plt.title('Train - Validation loss')
plt.figure()

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_width, img_height = 150, 150
img1 = image.load_img('/tmp/horse_or_human/horses/horse01-0.png', target_size = (img_width, img_height))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1, axis = 0)

img2 = image.load_img('/tmp/horse_or_human/humans/human01-00.png', target_size = (img_width, img_height))
img2 = image.img_to_array(img2)
img2 = np.expand_dims(img2, axis = 0)

print(model.predict(img1))
print(model.predict(img2))

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")
 

In [ ]:
import os
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img

successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

horse_img_files = [os.path.join(train_horse_path, f) for f in train_horse_names]
human_img_files = [os.path.join(train_human_path, f) for f in train_human_names]

img_path = random.choice(horse_img_files + human_img_files)

img = load_img(img_path, target_size=(150,150))
x = img_to_array(img)
x = x.reshape((1,) + x.shape)
x /= 255

successive_feature_maps = visualization_model.predict(x)
layer_names = [layer.name for layer in model.layers]

for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  if len(feature_map.shape) == 4:
    # Just do this for the conv / maxpool layers, not the fully-connected layers
    n_features = feature_map.shape[-1]  # number of features in feature map
    # The feature map has shape (1, size, size, n_features)
    size = feature_map.shape[1]
    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
      # Postprocess the feature to make it visually palatable
      x = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x += 128
      x = np.clip(x, 0, 255).astype('uint8')
      # We'll tile each filter into this big horizontal grid
      display_grid[:, i * size : (i + 1) * size] = x
    # Display the grid
    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)